In [71]:
tag2id, id2tag = {}, {} # maps tag to id. tag2id: {"VB": 0, "NNP":1, ...}, id2tag: {0: "VB", 1: "NNP"...}
word2id, id2word = {}, {} # maps word to id

for line in open("traindata.txt"):
    items = line.split('/')
    word, tag = items[0], items[1].rstrip() # 抽取每一个行里的单词和词性
    
    if word not in word2id:
        word2id[word] = len(word2id) # 如果word不在word2id中，就把它加进去然后赋予它一个下标，下标是长度即可
        id2word[len(id2word)] = word # 同上
        
    if tag not in tag2id:
        tag2id[tag] = len(tag2id)
        id2tag[len(id2tag)] = tag

M = len(word2id) # M: 词典的大小  # of word in dictionary
N = len(tag2id) # N: 词性的种类个数 # of tags in tag set
        
        

In [72]:
# 构建π, A, B, 先将他们初始化为0
import numpy as np
pi= np.zeros(N) # 每个词性出现在句子中第一个位置的概率。 # of tags
A = np.zeros((N, M)) # 给定A[i][j], 给定tag i, 出现单词 j 的概率。 N: # of tags, M: # of words in dictionary
B = np.zeros((N, N)) # 给定B[i][j], 之前的状态是 i, 转换成状态 j, 的概率

In [73]:
# 接下来就要在读取过程中将上边的信息一个一个的写入到 pi, A, B中

prev_tag = ""
for line in open('traindata.txt'):
    items = line.split('/')
    wordID, tagID = word2id[items[0]], tag2id[items[1].rstrip()] 
    
    if prev_tag == "": # 这意味着是句子的开始
        pi[tagID] += 1
        A[tagID][wordID] += 1
    else: # 如果不是句子的开头
        A[tagID][wordID] += 1
        B[tag2id[prev_tag]][tagID] += 1
        
    if items[0] == '.': # 意味着这个句子结束了
        prev_tag = ''
    else:
        prev_tag = items[1].rstrip()
        

In [74]:
# 我们前面算完了A, B, pi的次数, 现在我们考虑把次数转换成概率
# normalize
pi = pi/sum(pi)
for i in range(N):
    A[i] /= sum(A[i])
    B[i] /= sum(B[i])
# 到此为止我们计算完了模型的而所有的参数: pi, A, B

In [75]:
def log(v):
    if v == 0:
        return np.log(v+0.0000000000000000000000001)
    return np.log(v)

In [76]:
# 定义维特比算法
def viterbi(x, pi, A, B):
    """
    :param x: user input string/sentence 
    :param pi: initial probability of tags
    :param A: 给定tag，每个单词出现的概率
    :param B: 不同tag之间的状态转移概率
    :return: 
    """
    x = [word2id[word] for word in x.split(' ')] # x代表的是针对每个单词的序号
    T = len(x)
    dp = np.zeros((T, N)) # dp[i][j]: w1...wi, 假设wi的tag是第j个tag 
    
    for j in range(N): # base case for DP算法
        dp[0][j] = log(pi[j]) + log(A[j][x[0]]) 
        
    #　以下只是动态规划的结果，而我现在要记录下路径
    ptr = np.array([[0 for x in range(N)] for y in range(T)]) # 一个T*N的矩阵
    
    for i in range(1, T): # 每个单词
        for j in range(N): # 每个词性
            # TODO：以下几行的代码可以写成一行，（vectorize的操作，会使得效率变高）
            dp[i][j] = -99999999
            for k in range(N): # 从每一个k可以到达j
                score = dp[i-1][k] + log(A[j][x[i]]) + log(B[k][j])
                if score > dp[i][j]:
                    dp[i][j] = score  # 在这里我们可以看出，算法的时间复杂度是N平方乘以T
                    ptr[i][j] = k # 记录下每个点的上一个路径
    # decoding: 把最好的 tag sequence 打印出来
    best_seq = [0] * T   # best_seq = [1, 5, 2, 23, 4, ...]
    # step1: 找出对应于最后一个单词的词性
    best_seq[T-1] = np.argmax(dp[T-1]) # argmax求出最后一列的那个单词的最大的score
    # step2: 通过从后到前的循环来一次求出每个单词的词性
    # 注意：range函数(开始位置, 结束位置, 步长), 不包括结束位置的元素
    for i in range(T-2, -1, -1): # T-2, T-1, ..., 1, 0
        best_seq[i] = ptr[i+1][best_seq[i+1]]
    
    # 到目前为止，best_seq存放了对应于x的词性序列
    for i in range(len(best_seq)):
        print(id2tag[best_seq[i]])
        
    

In [77]:
x = 'the U.S. security burden in the region . The issue is further'
viterbi(x, pi, A, B)
 

DT
NNP
NN
NN
IN
DT
NN
.
DT
NN
VBZ
JJ
